<a href="https://colab.research.google.com/github/ChavesHygor/BairesDev---Machine-Learning-Training-ok/blob/main/Cria%C3%A7%C3%A3o_de_Uma_Base_de_Dados_e_Treinamento_da_Rede_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# 1. Verifica se a GPU está ativa e alocada para o notebook
!nvidia-smi

# 2. Monta o Google Drive para armazenamento persistente
from google.colab import drive
drive.mount('/content/gdrive')

# 3. Cria um link simbólico para a pasta darknet no seu Drive.
#    Isso evita erros com espaços no nome "My Drive".
#    Certifique-se de que a pasta 'darknet' existe na raiz do seu Drive.
!ln -s "/content/gdrive/My Drive/darknet" /mydrive

Tue Aug 26 16:37:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [53]:
# --- ETAPA 2: Clonar e Compilar o Darknet ---
import os

# 1. Clona o repositório do darknet, se ainda não foi clonado
if not os.path.exists('darknet'):
    print("Clonando o repositório do Darknet...")
    !git clone https://github.com/kriyeng/darknet/
else:
    print("Repositório Darknet já existe.")

# 2. Entra na pasta do darknet para poder executar os comandos
%cd /content/darknet

# 3. Edita o Makefile para habilitar GPU, CUDNN e OPENCV
print("\nConfigurando o Makefile...")
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile # Correção de compatibilidade com cuDNN do Colab

# 4. Instala a biblioteca de desenvolvimento do OpenCV
print("\nInstalando dependências do OpenCV...")
!apt-get -y install libopencv-dev

# 5. Cria o link simbólico para compatibilidade da versão do OpenCV
print("\nCriando link simbólico do OpenCV...")
!ln -sf /usr/lib/x86_64-linux-gnu/pkgconfig/opencv4.pc /usr/lib/x86_64-linux-gnu/pkgconfig/opencv.pc

# 6. Compila o darknet
print("\nCompilando o Darknet... (Isso pode levar alguns minutos)")
!make clean  # Limpa compilações anteriores
!make       # Compila o código

print("\nCompilação do Darknet concluída!")

Clonando o repositório do Darknet...
Cloning into 'darknet'...
remote: Enumerating objects: 10068, done.
remote: Total 10068 (delta 0), reused 0 (delta 0), pack-reused 10068 (from 1)
Receiving objects: 100% (10068/10068), 10.14 MiB | 16.33 MiB/s, done.
Resolving deltas: 100% (6715/6715), done.
/content/darknet

Configurando o Makefile...

Instalando dependências do OpenCV...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

Criando link simbólico do OpenCV...

Compilando o Darknet... (Isso pode levar alguns minutos)
rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_laye

In [54]:
# --- ETAPA 3: Preparação do Dataset ---
import os
import glob

# 1. Baixar o dataset de detecção de máscaras para a pasta /mydrive
dataset_path = "/mydrive/Face-Mask-Detection"
if not os.path.exists(dataset_path):
    print(f"Dataset não encontrado. Clonando para '{dataset_path}'...")
    !git clone https://github.com/datoun/Face-Mask-Detection.git {dataset_path}
else:
    print("Pasta do dataset já existe.")

# 2. Criar o arquivo obj.names com as nossas classes
#    (Usando %%writefile, que deve estar na PRIMEIRA linha da célula)
print("\nCriando arquivo obj.names...")

Dataset não encontrado. Clonando para '/mydrive/Face-Mask-Detection'...
Cloning into '/mydrive/Face-Mask-Detection'...
fatal: could not read Username for 'https://github.com': No such device or address

Criando arquivo obj.names...


In [55]:
%%writefile /mydrive/obj.names
with_mask
without_mask

Writing /mydrive/obj.names


In [56]:
%%writefile /mydrive/obj.data
classes = 2
train = /mydrive/train.txt
valid = /mydrive/test.txt
names = /mydrive/obj.names
backup = /mydrive/backup/

Writing /mydrive/obj.data


In [57]:
%%writefile /mydrive/obj.names
with_mask
without_mask

Overwriting /mydrive/obj.names


In [58]:
%%writefile /mydrive/obj.data
classes = 2
train = mydrive/train.txt
valid = mydrive/test.txt
names = mydrive/obj.names
backup = mydrive/backup

Overwriting /mydrive/obj.data


In [59]:
# --- ETAPA 3: Preparação do Dataset (VERSÃO FINAL COM VERIFICAÇÃO DE DOWNLOAD) ---
import os
import glob

print("--- Iniciando a Etapa 3: Preparação Completa do Dataset ---")

# --- 1. Definir caminhos e limpar dados antigos ---
dataset_dir = "/mydrive/FaceMaskDataset"
zip_file = "/content/dataset.zip"

if os.path.exists(dataset_dir):
    print(f"Removendo a pasta do dataset antigo: {dataset_dir}")
    !rm -rf {dataset_dir}

# --- 2. Baixar o Dataset com um novo link e com barra de progresso ---
print(f"\nBaixando o novo dataset como '{os.path.basename(zip_file)}'...")
# Este é um novo link, mais estável, para um dataset similar do Roboflow.
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_2aIBSf9Jm8h8g7e1e40w5f2s4OK29N5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_2aIBSf9Jm8h8g7e1e40w5f2s4OK29N5" -O {zip_file} && rm -rf /tmp/cookies.txt

# --- 3. VERIFICAR se o download foi bem-sucedido ---
print("\nVerificando o tipo do arquivo baixado...")
# Este comando deve retornar "Zip archive data". Se retornar "HTML document" ou outra coisa, o download falhou.
!file {zip_file}

# --- 4. Descompactar o Dataset ---
print(f"\nDescompactando o dataset para '{dataset_dir}'...")
# Usamos '&&' para executar o próximo comando apenas se o anterior for bem-sucedido
!unzip -q -o {zip_file} -d {dataset_dir} && echo "Dataset descompactado com sucesso!"

# --- 5. VERIFICAR se a descompactação funcionou ---
# Verificamos se um arquivo essencial existe antes de continuar
check_file = os.path.join(dataset_dir, "train/_darknet.txt")
if not os.path.exists(check_file):
    print("\nERRO CRÍTICO: A descompactação falhou ou o dataset não contém os arquivos esperados.")
    print(f"O arquivo '{check_file}' não foi encontrado.")
else:
    # --- 6. Criar o Arquivo obj.data ---
    print("\nCriando o arquivo de configuração obj.data...")
    with open("/mydrive/obj.data", "w") as f:
        f.write("classes = 2\n")
        f.write(f"train = {dataset_dir}/train/_darknet.txt\n")
        f.write(f"valid = {dataset_dir}/valid/_darknet.txt\n")
        f.write(f"names = {dataset_dir}/_darknet.labels\n")
        f.write("backup = /mydrive/backup/\n")

    # --- 7. Criar a Pasta de Backup ---
    !mkdir -p /mydrive/backup

    print("\n--- PASSO 3 CONCLUÍDO COM SUCESSO! ---")
    print("Todos os arquivos de configuração e listas de imagens foram criados a partir do novo dataset.")
    train_count = len(open(f"{dataset_dir}/train/_darknet.txt").readlines())
    valid_count = len(open(f"{dataset_dir}/valid/_darknet.txt").readlines())
    print(f"Imagens de treino: {train_count}")
    print(f"Imagens de validação/teste: {valid_count}")

--- Iniciando a Etapa 3: Preparação Completa do Dataset ---

Baixando o novo dataset como 'dataset.zip'...
--2025-08-26 16:37:55--  https://docs.google.com/uc?export=download&confirm=&id=1_2aIBSf9Jm8h8g7e1e40w5f2s4OK29N5
Resolving docs.google.com (docs.google.com)... 108.177.127.139, 108.177.127.138, 108.177.127.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.127.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1_2aIBSf9Jm8h8g7e1e40w5f2s4OK29N5&export=download [following]
--2025-08-26 16:37:55--  https://drive.usercontent.google.com/download?id=1_2aIBSf9Jm8h8g7e1e40w5f2s4OK29N5&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-08-26 16:37:56

In [45]:
# --- ETAPA 4: Configuração do Arquivo .cfg do Modelo ---
%cd /content/darknet

print("Configurando o arquivo yolov3_custom.cfg...")
# Copia o arquivo de configuração padrão
!cp cfg/yolov3.cfg /mydrive/yolov3_custom.cfg

# Altera o arquivo para 2 classes e ajusta parâmetros de treino
!sed -i 's/batch=64/batch=64/' /mydrive/yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=16/' /mydrive/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' /mydrive/yolov3_custom.cfg
!sed -i 's/steps=400000,450000/steps=3200,3600/' /mydrive/yolov3_custom.cfg
!sed -i 's/classes=80/classes=2/g' /mydrive/yolov3_custom.cfg
# A fórmula para os filtros é (classes + 5) * 3 = (2 + 5) * 3 = 21
!sed -i 's/filters=255/filters=21/g' /mydrive/yolov3_custom.cfg

print("Arquivo yolov3_custom.cfg configurado com sucesso!")

/content/darknet
Configurando o arquivo yolov3_custom.cfg...
Arquivo yolov3_custom.cfg configurado com sucesso!


In [46]:
# --- ETAPA 5: Treinamento ---
%cd /content/darknet

# 1. Baixa os pesos convolucionais pré-treinados (para transfer learning)
if not os.path.exists('darknet53.conv.74'):
    !wget https://pjreddie.com/media/files/darknet53.conv.74

# 2. Inicia o treinamento
print("\nINICIANDO O TREINAMENTO...")
!./darknet detector train /mydrive/obj.data /mydrive/yolov3_custom.cfg darknet53.conv.74 -dont_show -map

/content/darknet
--2025-08-26 16:32:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3037::6815:589c, ...
Connecting to pjreddie.com (pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74       [ <=>                ]   8.88K  --.-KB/s    in 0s      

2025-08-26 16:32:26 (97.3 MB/s) - ‘darknet53.conv.74’ saved [9093]


INICIANDO O TREINAMENTO...
/bin/bash: line 1: ./darknet: No such file or directory


In [48]:
# --- ETAPA 6: Teste do Modelo Treinado ---
import cv2
import matplotlib.pyplot as plt
import os
import glob

# Função para exibir imagens no Colab
def imShow(path):
    if not os.path.exists(path):
        print(f"ERRO: O arquivo de imagem de resultado '{path}' não foi encontrado.")
        print("Isso geralmente significa que o comando de detecção falhou.")
        return
    image = cv2.imread(path)
    if image is None:
        print(f"ERRO: O OpenCV não conseguiu ler o arquivo de resultado '{path}'.")
        return
    height, width = image.shape[:2]
    resized_image = cv2.resize(image, (3*width, 3*height), interpolation=cv2.INTER_CUBIC)
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()

# Garante que estamos no diretório correto para executar o darknet
%cd /content/darknet

# Recria a lista de arquivos de teste para garantir que a variável exista
all_images = glob.glob("/mydrive/Face-Mask-Detection/images/*.png") + glob.glob("/mydrive/Face-Mask-Detection/images/*.jpg")
test_files = all_images[int(len(all_images)*0.8):]

if not test_files:
    print("ERRO: Nenhum arquivo de teste encontrado. Execute a célula de preparação do dataset primeiro.")
else:
    # --- PONTO DE CONFIGURAÇÃO PRINCIPAL ---
    # Verifique na sua pasta /mydrive/backup/ o nome do arquivo de pesos que deseja testar
    weights_path = "/mydrive/backup/yolov3_custom_last.weights"

    # Seleciona a primeira imagem da lista de teste. Mude o índice para testar outras (ex: test_files[1])
    image_path_to_test = test_files[0]

    print(f"Usando os pesos: {weights_path}")
    print(f"Testando na imagem: {image_path_to_test}")

    # Verifica se o arquivo de pesos realmente existe
    if not os.path.exists(weights_path):
        print(f"\nERRO CRÍTICO: O arquivo de pesos '{weights_path}' não foi encontrado!")
        print("Verifique o nome do arquivo na sua pasta '/mydrive/backup/' e corrija a variável 'weights_path'.")
    else:
        # Roda o comando de detecção
        !./darknet detector test /mydrive/obj.data /mydrive/yolov3_custom.cfg {weights_path} {image_path_to_test} -thresh 0.3

        # Exibe o resultado, que é salvo como 'predictions.jpg'
        print("\n--- Resultado da Detecção ---")
        imShow('predictions.jpg')

/content/darknet
ERRO: Nenhum arquivo de teste encontrado. Execute a célula de preparação do dataset primeiro.
